In [ ]:
struct LinearAttention{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [ ]:
Flux.functor(mh::LinearAttention) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> LinearAttention(mh.head, mh.future, m..., mh.drop)


In [ ]:
LinearAttention_1(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = LinearAttention(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::LinearAttention)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "LinearAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end

In [ ]:
function (mh::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}

# of type float (to allow for integer inputs)
oftf(x, y) = oftype(float(x), y)
nelu(x, α=1) = ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)

In [ ]:
function splitHeads(x, batch, head, depth)
    x = reshape(x, (batch, -1, head, depth))
end    

In [1]:
function (mh::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

ipq = nelu(splitHeads(ipq, qs[1], mh.head, hs))
ipk = nelu(splitHeads(ipk, qs[1], mh.head, hs))
ipv = nelu(splitHeads(ipv, qs[1], mh.head, hs))

@einsum kv[m,d,e,h] = ipk[m,j,h,d]*ipv[m,j,h,e]

sum_1 = sum(ipk, axis=1) + 1e-8

@einsum z_1[m,l,h] = ipq[m,l,h,d]*sum_1[m,h,d]

z = 1/z_1

@einsum output[m,l,h,e] = ipq[m,l,h,d]*kv[m,d,e,h]*z[m,l,h]

output = reshape(output, (qs[1], -1, mh.head*hs))

return output

end



LoadError: LoadError: UndefVarError: @toNd not defined
in expression starting at Untitled-1.ipynb:13